## Imports

In [1]:
from os import path, listdir, walk
import datetime

import numpy as np
import pandas as pd
from timeit import default_timer as timer
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, brier_score_loss
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertTokenizer

## Read data


In [2]:
ENCODED_DATA_PATH = '/content/tokenized'
RAW_DATA_PATH = '/content/text'
BATCH_SIZE = 32
MAX_SEQ_SIZE = 768
TEST_SET_FRACTION = 0.3

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
tokenized_df = pd.read_pickle('/content/tokenized_texts.pkl')

In [6]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [7]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return accuracy_score(y_true, y_pred)

def calculate_f1(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return f1_score(y_true, y_pred)

def calculate_brier(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)
    return brier_score_loss(y_true, y_prob)

def calculate_auc(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)

    false_positive_rates, true_positive_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(false_positive_rates, true_positive_rates)

    return roc_auc, false_positive_rates, true_positive_rates

In [8]:
def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()
    train_loss = 0
    steps = 0

    for batch in dataloader:
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        y_hat = model(b_input_ids,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = y_hat.loss
        train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        steps += 1

    # Calculate the average loss over all of the batches.
    return train_loss / steps

In [20]:
def test_step(model, dataloader, loss_fn, device):
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    steps = 0

    all_y_true = []
    all_y_hat = []

    # Evaluate data for one epoch
    for batch in dataloader:
        steps += 1
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.inference_mode():
            y_hat = model(b_input_ids,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = y_hat.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = y_hat.logits.detach().cpu()
        label_ids = b_labels.detach().cpu().reshape(-1,1)
        logits = torch.max(logits, dim=1, keepdim=True).values
        all_y_true.extend(label_ids)
        all_y_hat.extend(logits)

    all_y_true = torch.cat(all_y_true, dim=0)
    all_y_hat = torch.cat(all_y_hat, dim=0)
    # Calculate the average loss over all of the batches.
    test_accuracy = flat_accuracy(all_y_true, all_y_hat)
    test_f1 = calculate_f1(all_y_true, all_y_hat)
    test_brier = calculate_brier(all_y_true, all_y_hat)
    test_auc_tuple = calculate_auc(all_y_true, all_y_hat)

    return total_eval_loss / steps, test_accuracy, test_f1, test_brier, test_auc_tuple


In [10]:
def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "test_loss": [],
        "test_acc": [],
        "test_f1": [],
        "test_brier": [],
        "test_auc_tuple": []
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc, test_f1, test_brier, test_auc_tuple = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["test_f1"].append(test_f1)
        results["test_brier"].append(test_brier),
        results["test_auc_tuple"].append(test_auc_tuple)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"test_f1: {test_f1:.4f} | "
            f"test_brier: {test_brier:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

In [11]:
def get_tensor_dataset_from_df(df):
    input_ids = torch.cat(list(df['input_ids']), dim=0)
    attention_masks = torch.cat(list(df['attention_mask']), dim=0)
    labels = torch.tensor(list(df['labels']))

    return TensorDataset(input_ids, attention_masks, labels)

In [12]:
def test_against_all(model, trained_llm_name, human_test_df, llm_df, loss_fn, device):
    all_results = []

    for dataset_name in llm_df['llm_name'].unique():
        curr_llm_df = llm_df.loc[llm_df['llm_name'] == dataset_name]
        test_df = pd.concat([human_test_df, curr_llm_df], ignore_index=True)

      #  train_dataset = get_tensor_dataset_from_df()
        test_dataset = get_tensor_dataset_from_df(test_df)

        test_dataloader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=True,
        )

        results = test_step(
            model,
            test_dataloader,
            loss_fn,
            device
        )

        all_results.append({
            dataset_name: results
        })

    return all_results

In [13]:
human_train_df, human_test_df = train_test_split(tokenized_df.loc[tokenized_df['llm_name'] == 'human'], test_size=TEST_SET_FRACTION, random_state=69)


In [ ]:
final_results = []

for llm_name in tokenized_df['llm_name']:
    llm_df = tokenized_df.loc[tokenized_df['llm_name'] == llm_name]

    llm_train_df, llm_test_df = train_test_split(llm_df, test_size=TEST_SET_FRACTION, random_state=69)
    train_df = pd.concat([human_train_df, llm_train_df], ignore_index=True)
    test_df = pd.concat([human_test_df, llm_test_df], ignore_index=True)

    train_dataset = get_tensor_dataset_from_df(train_df)
    test_dataset = get_tensor_dataset_from_df(test_df)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=True,
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=True,
    )

    # Load BertForSequenceClassification, the pretrained BERT model with a single
    # linear classification layer on top.
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-cased",
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    if device == "cuda:0":
      model = model.cuda()

    model = model.to(device)
    optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

    epochs = 3

    loss_fn = torch.nn.BCEWithLogitsLoss()

    print(f'Training model for {llm_name}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=10,
        device=device
    )

    print(f'Finished training model for {llm_name}')

    print(f'Testing against all for {llm_name}...')

    results_against_all = test_against_all(
        model,
        llm_name,
        human_test_df,
        tokenized_df,
        loss_fn,
        device
    )

    final_results.append({
        'results_against_itself': current_results,
        'results_against_all': results_against_all
    })

    print(f'Finished testing against all for {llm_name}')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training model for mistralai-mistral-7b-instruct-v0.2...


 10%|█         | 1/10 [01:18<11:45, 78.42s/it]

Epoch: 1 | train_loss: 0.3405 | test_loss: 0.3316 | test_acc: 0.5016 | test_f1: 0.6624 | test_brier: 0.3734 | time: 67.2572


 20%|██        | 2/10 [02:39<10:39, 79.98s/it]

Epoch: 2 | train_loss: 0.1561 | test_loss: 0.5861 | test_acc: 0.4953 | test_f1: 0.6596 | test_brier: 0.4346 | time: 69.5310


In [ ]:
t = tokenized_df.head(1)
print(t['input_ids'].shape, t['attention_mask'].shape, t['labels'].shape)
t1 = get_tensor_dataset_from_df(t)
t1